## <span style="color:red">Class (as) decorators</span>

Come abbiamo visto, la presenza del metodo \__call\__ nella classe ne rende gli oggetti "callable". In quanto tali, essi possono essere utilizzabili come decoratori. 

Come (utile) esempio per descrivere questi decoratori consideriamo il problema di generare numeri (pseudo)casuali la cui funzione di (densità) di probabilità sia un'esponenziale negativa:
$$
f(x;\lambda) = \left\{\begin{matrix}
\lambda e^{-\lambda x} & x \ge 0, \\
0 & x < 0.
\end{matrix}\right.
$$
dove $\lambda>0$ è un parametro che può essere interpretato come *intensità* (di un fenomeno) o *frequenza* (di un evento).

L'esponenziale negativa è una distribuzione **fondamentale** in esperimenti di simulazione al computer.

La funzione di distribuzione di probabilità dell'esponenziale negativa di parametro $\lambda$ è $F(x;\lambda)=1-e^{-\lambda x}$ e si può dimostrare che, se $\{r_n\}$ è una sequenza di numeri casuali uniformemente distribuiti in $[0,1)$ allora $\{F^{-1}(r_n)\}$ è una sequenza distribuita con esponenziale negativa. Risulta poi facilmente:
$$F^{-1}(r) = -\frac{\ln(1-r)}{\lambda}$$

Scriviamo ora una classe Python che implementi un generatore di numeri distribuiti con esponenziale negativa

In [ ]:
from math import log
class negexp:
    '''Classe che implementa un generatore uniforme che produce
       numeri distribuiti con esponenziale negativa (detti anche,
       con terminologia inglese, "negative exponential deviate") di
       parametro lambda assegnato.'''
    
    def __init__(self, LAMBDA):
        '''LAMBDA (scritto maiuscolo perché la 
           forma minuscola è una parola riservata di Python)
           è il parametro della distribuzione .'''
        self._lambda = LAMBDA

    def __call__(self, fn):
        '''fn è il generatore uniforme'''
        def decorated():
            r = fn()
            return -log(1.0-r)/self._lambda
        return decorated

Possiamo usare il generatore uniforme "standard" di Python...

In [ ]:
from random import random

In [ ]:
@negexp(0.5)
def schedule():
    return random()

In [ ]:
schedule() # z.s. per negexp(0.5).__call__(random)()

In [ ]:
class myrand:
    '''Un semplice generatore pseudo-casuale'''
    def __init__(self):
        from time import time
        self._a = 16807               # a = 16807
        self._m = (1 << 31) - 1       # m = 2^31 -1
        self._d = 1.0/self._m
        self._x = int(time())%self._m # x_0 "casuale", dipendente dal tempo
        if self._x==0:
            self._x = 1               # ... ma non 0
        for i in range(10000):  # Facciamo un po' di giri "a vuoto" per rendere i numeri
                                # (apparentemente) indipendenti dal seme iniziale
            self.__call__()
    
    def __call__(self):
        self._x = (self._a*self._x)%self._m
        return self._x*self._d
    
r = myrand()

In [ ]:
LAMBDA = 0.2
@negexp(LAMBDA)
def schedule():
    return r()

In [ ]:
r()

Facciamo una piccola verifica. Per t (tempo) sufficientemente grande, vediamo quandi eventi vengono "schedulati" nelle prossime t "unità di misura" (millisecondi, secondi, ore, ...)

In [ ]:
# Consideriamo t unità di tempo (con t crescente)
for i in range(1,9):
    t = 10**i
    count = s = 0
    while s<t:
        s += schedule()
        count += 1
    print(f'10^{i}\t',count/t)  # Il secondo valore, al crescere di t, deve avvicinarsi a LAMBDA

### Decorare una classe

Finora abbiamo visto l'uso di una classe per decorare una funzione. Esiste però anche la possibilità di <u>decorare una classe</u>. 

Ci sono in realtà due modi diversi per decorare classi: il primo è di decorare i suoi (o solo alcuni) metodi; il secondo consiste invece nel decorare l'intera classe.

#### Decoratori built-in per metodi

1. @property. Abbiamo già visto questo decoratore: esso viene utilizzato per specializzare l'accesso in lettura e scrittura a particolari attributi della classe
2. @classmethod. Un metodo decorato in questo modo non viene invocato sulle istanze di una classe. Esso viene piuttosto utilizzato per creare istanze "specializzate" di oggetti della classe. Un esempio è illustrato di seguito
3. @staticmethod. Un metodo statico può essere chiamato utilizzando sia la classe sia un oggetto della classe perché in realtà è legato alla classe solo per il fatto di essere parte del suo namespace. Un tipico esempio solo i metodi di una "libreria" matematica

In [ ]:
from math import log
class negexp:
    '''Classe che implementa un generatore uniforme che produce
       numeri distribuiti con esponenziale negativa (detti anche,
       con terminologia inglese, "negative exponential deviate") di
       parametro lambda assegnato.'''
    
    def __init__(self, LAMBDA):
        '''LAMBDA (scritto maiuscolo perché la 
           forma minuscola è una parola riservata di Python)
           è il parametro della distribuzione .'''
        self._lambda = LAMBDA
        
    @classmethod
    def unitfreq(cls):
        return cls(1.0) 

    def __call__(self, fn):
        '''fn è il generatore uniforme'''
        #self.rand = fn
        def decorated():
            #r = self.rand()
            r = fn()
            return -log(1.0-r)/self._lambda
        return decorated

In [ ]:
from random import random
@negexp.unitfreq()
def schedule():
    return random()

In [ ]:
# Ripetiamo il "controllo"
for i in range(1,9):
    t = 10**i
    count = s = 0
    while s<t:
        s += schedule()
        count += 1
    print(f'10^{i}\t',count/t)  # Il secondo valore deve avvicinarsi a 1 per t crescente

#### Decoratori "user-defined" per metodi
Naturalmente si possono utilizzare anche decoratori che non siano built-in

In [ ]:
class look_and_say_seq:
    '''Restituisce un iterabile per la sequenza dei primi n numeri
       della c.d. serie "look-and-say" (o "look-and-read").
       I primi 6 tali numeri sono: 1, 11, 21, 1211, 111221, 312211
    '''
    def __init__(self,n):
        if n==0:
            self._stop = True
        else:
            self._stop = False
        self._next = '1'
        self._count = n
        
    def __iter__(self):
        return self
    
    def __next__(self):
        if self._stop:
            raise StopIteration
        nextval = self._next[:]
        self._next = ''
        i = j = 0
        l = len(nextval)
        while j<l:
            f = nextval[i]
            while j<l and nextval[j]==f:
                j += 1
            self._next += f'{j-i}{f}'
            i = j
        self._count -= 1
        self._stop = self._count == 0
        return int(nextval)

In [ ]:
L = look_and_say_seq(10)

In [ ]:
for s in look_and_say_seq(15):
    print(s)

In [ ]:
def my_timer(f):
    from time import time
    def timedfun(n):
        start_time = time()
        res = f(n)
        stop_time = time()
        elapsed_time = round((stop_time-start_time)*1000000)
        print(f"Elapsed time: {elapsed_time}\u03bcs")
        return res
    return timedfun

In [ ]:
class look_and_say_seq:
    '''Restituisce un iterabile per la sequenza dei primi n numeri
       della c.d. serie "look-and-say" (o "look-and-read").
       I primi 6 tali numeri sono: 1, 11, 21, 1211, 111221, 312211
    '''
    def __init__(self,n):
        if n==0:
            self._stop = True
        else:
            self._stop = False
        self._next = '1'
        self._count = n
        
    def __iter__(self):
        return self
    
    @my_timer
    def __next__(self):
        if self._stop:
            raise StopIteration
        nextval = self._next[:]
        self._next = ''
        i = j = 0
        l = len(nextval)
        while j<l:
            f = nextval[i]
            while j<l and nextval[j]==f:
                j += 1
            self._next += f'{j-i}{f}'
            i = j
        self._count -= 1
        self._stop = self._count == 0
        return int(nextval)

In [ ]:
for s in look_and_say_seq(20):
    print(s)

#### Decorazione di un'intera classe

Vediamo un esempio semplice, senza alcun rilievo applicativo

In [ ]:
def classdec(cls):
    '''In questo caso, il nuovo metodo è "inserito" direttamente nella classe'''
    def newmethod(self, x):
        print(f"newmethod printing {x}")
    cls.newmethod = newmethod 
    return cls

@classdec
class A:
    def __init__(self):
        print(f"Initializing class {self.__class__}")
    def oldmethod(self,x):
        print(f"oldmethod printing {x}")

In [ ]:
a = A()
a.oldmethod('foo')
a.newmethod('fie')

In [ ]:
def classdec(cls):
    '''In questo secondo caso la decorazione avviene invece 
       sostituendo la classe originale con una sottoclasse
       che "ospita" il nuovo metodo'''
    class subcls(cls):
        def newmethod(self, x):
            print(f"newmethod printing {x}")
    return subcls

@classdec
class B:
    def __init__(self):
        print(f"Initializing class {self.__class__}")
    def oldmethod(self,x):
        print(f"oldmethod printing {x}")

In [ ]:
b = B()
b.oldmethod('foo')
b.newmethod('fie')